In [1]:
import pandas as pd
from graphviz import Digraph
import numpy as np

In [2]:
data = {
    "Origin": ["S0", "S1", "S1", "S2"],
    "Action": ["NA", "a", "b", "NA"],
    "S0": [0.1, 0.2, 0.4, 0.3],
    "S1": [0.5, 0.3, 0.4, 0.3],
    "S2": [0.4, 0.5, 0.2, 0.4]
}

# Criando o DataFrame
df = pd.DataFrame(data)

# Exibindo o DataFrame
display(df)


,Origin,Action,S0,S1,S2
0,S0,NA,0.1,0.5,0.4
1,S1,a,0.2,0.3,0.5
2,S1,b,0.4,0.4,0.2
3,S2,NA,0.3,0.3,0.4


In [3]:
# Cria um objeto Graphviz para o gráfico direcionado
dot = Digraph(comment='MDP')

# Ajusta o tamanho do gráfico e configurações de layout
dot.attr(size='15,15')  # Aumenta o tamanho do gráfico
dot.attr('node', shape='circle', width='1', height='1', fontsize='12')  # Ajusta o tamanho dos nós
dot.attr('edge', fontsize='10', style='solid')  # Ajusta estilo das arestas para sólido, aumenta a fonte

# Adiciona os nós (estados)
for state in df['Origin'].unique():
    dot.node(state, state)

# Adiciona as arestas (transições) sem diferenciar por ação
for index, row in df.iterrows():
    origin = row['Origin']
    for dest in ['S0', 'S1', 'S2']:
        if row[dest] > 0:
            # Ação é incluída na label se não for "NA"
            label = f"{row['Action']}, {row[dest]}" if row['Action'] != "NA" else f"{row[dest]}"
            dot.edge(origin, dest, label=label)

# Renderiza o gráfico em um arquivo
dot.render('mdp_graph_updated', format='png', view=True)



'mdp_graph_updated.png'

In [4]:
def random_walk(df, start_state, steps, action='a'):
    current_state = start_state
    path = [current_state]
    
    for _ in range(steps):
        # Filtra as transições possíveis para o estado atual e a ação (incluindo "NA")
        transitions = df[(df['Origin'] == current_state) & ((df['Action'] == action) | (df['Action'] == "NA"))]
        
        # Se não houver transições válidas, termina o loop
        if transitions.empty:
            print("No valid transitions from state", current_state)
            break
        
        # Assegura que estamos selecionando apenas as colunas de estados para as probabilidades
        state_columns = [col for col in df.columns if col.startswith('S')]
        probabilities = transitions[state_columns].values.flatten()
        
        # Verifica se a soma das probabilidades é 1, se não, ajusta
        probabilities = probabilities / probabilities.sum()
        
        # Escolhe o próximo estado baseado nas probabilidades
        next_state = np.random.choice(state_columns, p=probabilities)
        
        path.append(next_state)
        current_state = next_state
    
    return path

In [5]:

# Executa um random walk a partir do estado S0, com 10 passos, e ação 'a'
random_walk_path = random_walk(df, 'S0', 2, 'a')
random_walk_path

['S0', 'S2', 'S1']